# 1

#### En el ejemplo de teoría hemos visto la forma de entrenar utilizando un trozo de datos como en la imagen "way1.png". 

(Básicamente, tener en cuenta más datos de entrenamiento [[20% de test]])

Investiga la forma de realizar el ejercicio a partir del gif "way3_sliding_window.gif":

- ¿Da mejores resultados entrenar el modelo así?

Algo muy típico en el trabajo de data scientist

![](../rnn/sliding_windows_google.png)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import Callback

In [2]:
# EN FUNCIONES

In [3]:
# Total time points
N = 3000
# First partition
P = int(N/3)
# Train/test splits per partition
Tp = int(0.8*P) 


# Second partition - train
P2 = int(1.8*P)

# Second partition - train/test
Tp2 = P

# Third partition - train
P3 = 3000

# Third partition - train/test
Tp3 = P2


t=np.arange(0,N)
x=(2*np.sin(0.02*t)*np.sin(0.003*t))+0.5*np.random.normal(size=N)
df = pd.DataFrame(x, columns=['Data'])


In [4]:
print(f"All data: {N}")
print(f"Data 1st partition: {P}")
print(f"Train data 1st partition: {Tp}\n")

print(f"Data 2nd partition: {P2}")
print(f"Train data 2nd partition: {Tp2}\n")

print(f"Data 3rd partition: {P3}")
print(f"Train data 3rd partition: {Tp3}\n")

All data: 3000
Data 1st partition: 1000
Train data 1st partition: 800

Data 2nd partition: 1800
Train data 2nd partition: 1000

Data 3rd partition: 3000
Train data 3rd partition: 1800



In [5]:
# All steps are done in small functions to then incorporate them to a larger one at the end. 
# This main function will return the accuracy of each iteration.

In [6]:
def train_test_split(Tp, P):
    # train/test per partition
    train, test = df.values[0:Tp ,:], df.values[Tp:P,:]
    return train, test

In [7]:
train, test = train_test_split(Tp, P)

In [8]:
# step
def step (n_step, test, train):

    # add step elements into train and test
    test = np.append(test, np.repeat(test[-1,], n_step))
    train = np.append(train, np.repeat(train[-1,], n_step))
    return train, test

In [9]:
train, test = step(4, test, train)

In [10]:
# converting to a multi-dimensional array
def convertToMatrix(data, data1, n_step):
    X, Y =[], []
    X1,Y1 = [], []
    # train
    for i in range(len(data)-n_step):
        d=i+n_step  
        X.append(data[i:d,])
        Y.append(data[d,])
    # test
    for i in range(len(data1)-n_step):
        d=i+n_step  
        X1.append(data1[i:d,])
        Y1.append(data1[d,])
        
    return np.array(X), np.array(Y), np.array(X1), np.array(Y1)

In [11]:
n_step = 4
trainX,trainY, testX, testY = convertToMatrix(train, test, n_step)

In [12]:
# reshape

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [13]:
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM

def build_simple_rnn(num_units=128, embedding=4, num_dense=32, lr=0.001):    #embedding = step, lr=learning rate
    """
    Builds and compiles a simple RNN model
    Arguments:
              num_units: Number of units of a the simple RNN layer
              embedding: Embedding length
              num_dense: Number of neurons in the dense layer followed by the RNN layer
              lr: Learning rate (uses RMSprop optimizer)
    Returns:
              A compiled Keras model.
    """
    model = Sequential()
    # Long short term memory
    model.add(LSTM(units=num_units, input_shape=(1, embedding), activation="relu"))
    model.add(Dense(num_dense, activation="relu"))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=RMSprop(learning_rate=lr),metrics=['mse'])
    
    return model

In [14]:
class MyCallback(Callback):
        def on_epoch_end(self, epoch, logs=None):
            if (epoch+1) % 50 == 0 and epoch>0:
                print("Epoch number {} done".format(epoch+1))

In [15]:
batch_size=16
num_epochs = 1000

def model_training(batch_size, num_epochs, trainX, trainY, model):
    model.fit(trainX,trainY, 
            epochs=num_epochs, 
            batch_size=batch_size, 
            callbacks=[MyCallback()],verbose=0)

In [16]:
def train_partition(Tp, P, n_step, data, data1, num_units, embedding, num_dense, lr, batch_size, num_epochs):
    train, test = train_test_split(Tp, P)
    train, test = step(n_step, test, train)
    
    
    trainX,trainY, testX, testY = convertToMatrix(data, data1, n_step)
    # reshape

    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

    # model
    model = build_simple_rnn(num_units, embedding, num_dense, lr)

    class MyCallback(Callback):
        def on_epoch_end(self, epoch, logs=None):
            if (epoch+1) % 50 == 0 and epoch>0:
                print("Epoch number {} done".format(epoch+1))
        
    model_training(batch_size, num_epochs, trainX, trainY, model)
    #Accuracy
    test_loss, test_acc = model.evaluate(testX, testY, verbose=2)
    return ('\nTest accuracy:', test_acc)

In [17]:
# 1st partition
acc1 = train_partition(Tp, P, n_step=4, data=train, data1=test, num_units=128, embedding=4, num_dense=32, lr=0.001, batch_size=16, num_epochs=1000)
acc1

Epoch number 50 done
Epoch number 100 done
Epoch number 150 done
Epoch number 200 done
Epoch number 250 done
Epoch number 300 done
Epoch number 350 done
Epoch number 400 done
Epoch number 450 done
Epoch number 500 done
Epoch number 550 done
Epoch number 600 done
Epoch number 650 done
Epoch number 700 done
Epoch number 750 done
Epoch number 800 done
Epoch number 850 done
Epoch number 900 done
Epoch number 950 done
Epoch number 1000 done
7/7 - 0s - loss: 0.6485 - mse: 0.6485


('\nTest accuracy:', 0.6485034227371216)

In [18]:
# 2nd partition
acc2 = train_partition(Tp2, P2, n_step=4, data=train, data1=test, num_units=128, embedding=4, num_dense=32, lr=0.001, batch_size=16, num_epochs=1000)
acc2

Epoch number 50 done
Epoch number 100 done
Epoch number 150 done
Epoch number 200 done
Epoch number 250 done
Epoch number 300 done
Epoch number 350 done
Epoch number 400 done
Epoch number 450 done
Epoch number 500 done
Epoch number 550 done
Epoch number 600 done
Epoch number 650 done
Epoch number 700 done
Epoch number 750 done
Epoch number 800 done
Epoch number 850 done
Epoch number 900 done
Epoch number 950 done
Epoch number 1000 done
7/7 - 0s - loss: 0.6480 - mse: 0.6480


('\nTest accuracy:', 0.6480241417884827)

In [19]:
# 3rd partition
acc3 = train_partition(Tp3, P3, n_step=4, data=train, data1=test, num_units=128, embedding=4, num_dense=32, lr=0.001, batch_size=16, num_epochs=1000)
acc3

Epoch number 50 done
Epoch number 100 done
Epoch number 150 done
Epoch number 200 done
Epoch number 250 done
Epoch number 300 done
Epoch number 350 done
Epoch number 400 done
Epoch number 450 done
Epoch number 500 done
Epoch number 550 done
Epoch number 600 done
Epoch number 650 done
Epoch number 700 done
Epoch number 750 done
Epoch number 800 done
Epoch number 850 done
Epoch number 900 done
Epoch number 950 done
Epoch number 1000 done
7/7 - 0s - loss: 0.6114 - mse: 0.6114


('\nTest accuracy:', 0.6113775372505188)

In [20]:
import pandas as pd 
list_acc = [acc1[1], acc2[1], acc3[1]]
df = pd.DataFrame(list_acc, columns=["Accuracy"])
df

,Accuracy
0,0.648503
1,0.648024
2,0.611378
